# Eksempel på koder vi har kjørt for å teste ut vårt semantiske rom:


#!/usr/bin/env python3
 -*- coding: utf-8 -*-
"""
Created on Tue Oct 25 12:29:21 2022
 
@author: ketil
"""
#%%

In [ ]:
#Mange pakker nødvendig, også for nltk
from gensim.models import Word2Vec, KeyedVectors
import pandas as pd
import numpy as np
import nltk
import sqlite3
import matplotlib.pyplot as plt
nltk.download('punkt')
from sklearn.manifold import TSNE
import random
#%%
 
#%%
import os
os.getcwd()
os.chdir('/Users/ketil/Documents/Programming tools and tests/Gensim codes')
#%%
  
#%%
#Importerer datasett fra Github og lagrer det lokalt
import requests
 
url = 'https://raw.githubusercontent.com/robinjansson/puls-r_forskningsprosjekt/d6a33a5ba29508ad36bd14785e0a8b591aa38849/norsk_aviskorpus_clean_nps_10k.csv?token=GHSAT0AAAAAAB2CTZPXO27NU536IITV2YWAY2Y54LQ'
r = requests.get(url, allow_redirects=True)
 
open('data.csv', 'wb').write(r.content)
#%%
 
#%%
df = pd.read_csv('norsk_aviskorpus_clean_nps.csv', delimiter='/n')
 
#%%
 
#%%
df.Article.head(10)
articles = df.Article.values
print(f'{len(articles):,}')
#%%
 
#%%
articles[1003]
 
#%%
 
#%%
articleVec = [nltk.word_tokenize(Article) for Article in articles]
#%%
 
#%%
#Check if the tokenization worked
articleVec[6]
#%%
 
#%%
#Bestemmer lengden på ord-databasen:
training_words = []
for i in range (len(articleVec)):
    training_words.append(len(articleVec[i]))
 
print(f'Antall tokens: {sum(training_words):,}')
print(f'Gjennomsnittlig dokumentlengde: {sum(training_words) / len(articles):,.0f}')
#%%
 
#%%
#Trener Word2Vec-modell, dvs modell:
    model = Word2Vec(articleVec, vector_size=100, workers=4)
    model.wv[0]
    model.save("word2vec_model_NoAv_TEST_vs100")
#%%
 
#%%
#Henter ut alle ordene fra det semantiske rommet (model.wv.index_to_key)
sr_words = model.wv.index_to_key
print(f'Antall ord i semantisk rom: {len(sr_words):,}')
#%%
 
#%%
# Henter ut alle vektorene i det semantiske rommet
sr_vectors = model.wv.vectors
print(f'Antall vektorer i semantisk rom: {len(sr_vectors):,}')
#%%
 
#%%
#Finner "word count" for for definert ord
model.wv.get_vecattr('test', 'count')
#%%
 
#%%
#Hyppigste og sjeldneste ord:
    #Først en ordtelling:
word_count = []
for i in range(len(sr_words)):
    count = model.wv.get_vecattr(sr_words[i], 'count')
    word_count.append(count)    
#%%
 
#%%
#Deretter de 20 hyppigste ord:
def word_count_top(num):
    top_words = sr_words[0:num]
    top_count = word_count[0:num]
    
    for i in range(len(top_words)):
        print(f'{top_words[i]}: {top_count[i]}')
        
word_count_top(20)
#%%
 
#%%
#Detter de minst frekvente ordene:
def word_count_low(num):
    low_words = sr_words[-num:]
    low_count = word_count[-num:]
    
    for i in range(len(low_words)):
        print(f'{low_words[i]}: {low_count[i]}')
        
word_count_low(20)    
#%%
 
#%%
#Visualisering av "word count" for alle ord i semantisk rom
%matplotlib inline
plt.rcParams['figure.figsize'] = [5, 5]
plt.plot(word_count)
plt.show()
#%%
 
#%%
#Visualisering av "word count" for top 500 ord i semantisk rom
%matplotlib inline
plt.rcParams['figure.figsize'] = [5, 5]
plt.plot(word_count[0:300])
plt.show()
#%%
 
#%%
#Antall ord over / under et definert "word count"
def word_count_num(num):
    below_count = 0
    for i in range(len(word_count)):
        if word_count[i] < num:
            below_count += 1
    above_count = len(sr_words) - below_count
 
    print('Above {}: {}'.format(num, above_count))
    print('Below {}: {}'.format(num, below_count))
 
word_count_num(10) #definer "word count"
#%%
 
#%%
#Top x "most similar" ord til et definert ord
model.wv.most_similar('overrasket', topn=10)
 
#%%
 
#%%
#Tweak for å hente ut hele listen med "most similar" (topn=sys.maxsize)
import sys
all_sims = model.wv.most_similar('angst', topn=sys.maxsize)
len(all_sims)
 
all_sims[0]
#%%
 
#%%
#Funksjon for å beregne "least similar" 
def least_similar(word, lastn=5):
    all_sims = model.wv.most_similar(word, topn=sys.maxsize)
    least_sim = list(reversed(all_sims[-lastn:]))
    return least_sim
 
least_similar('angst', lastn=10)
 
 
#%%
 
#%%
#model.wv.similarity
model.wv.similarity('positiv', 'negativ')
 
#%%
 
#%%
#Funksjon for å beregne "cosine similarity" fra scratch
 
def cosine_similarity(word_1, word_2):
    
    for i in range(len(sr_words)):
        if sr_words[i] == word_1:
            v1 = sr_vectors[i]
        if sr_words[i] == word_2:
            v2 = sr_vectors[i]
    
    v1_norm = v1 / np.linalg.norm(v1)
    v2_norm = v2 / np.linalg.norm(v2)
    
    cos_sim = np.dot(v1_norm, v2_norm)
    
    return cos_sim  
 
cosine_similarity('positiv', 'negativ')
#%%
 
#%%
#model.wv.doesnt_match
#Ord som passer minst inn blant en liste med ord:@
testlist = ['angst', 'depresjon', 'søvnløshet', 'smile']
model.wv.doesnt_match(testlist)
 
#%%
 
#%%
#Eksempel med en hel fasett
con = sqlite3.connect('analysebibliotek.db')
cur = con.cursor()
cur.execute('''SELECT WORD FROM Norsk_Database_V3 WHERE lower(FACET)=?;''',('tillit', ))
f = cur.fetchall()
con.commit()
con.close()
 
facet = []
for i in range(len(f)):
    facet.append(f[i][0].lower())
 
#%%
 
 
